In [ ]:
import numpy as np
import os
import shutil
import pandas as pd
import sys
import gzip
import nibabel as nib
from matplotlib import pyplot as plt
from scipy import stats
import copy
import re

## Import Files

In [ ]:
path = '/Users/nanatang/VAserversim/m/Researchers/SerenaT/fromjapan_fullruns/'

In [ ]:
reportfile = 'grand_PVS_report.csv'
report = pd.read_csv(os.path.join(path,reportfile))

reportrawfile = 'grand_PVS_report_RAW.csv'
reportraw = pd.read_csv(os.path.join(path,reportrawfile))

demofile = 'idaSearch_2_21_2024-ADNI3.csv'
demo = pd.read_csv(os.path.join(path,demofile))

imagelistfile = 'imagelist_adni3frangi_T1only.csv'
imagelist = pd.read_csv(os.path.join(path,imagelistfile))

imagelistfsdnfile = 'imagelist_adni3fsdn_T1only.csv'
imagelistfsdn = pd.read_csv(os.path.join(path,imagelistfsdnfile))

dxfile = 'dx_adni3.csv'
dx = pd.read_csv(os.path.join(path,dxfile))

faultysubjectfile = 'faulty_subjects.csv'
faultysubjects = pd.read_csv(os.path.join(path,faultysubjectfile),names=['subjects'])

## Missing subjects investigation
- why doesn't the fsdn image list match the frangi image list and which ones don't match
- which subjects that exist in the adni3_frangi list are missing years in the grand report
- any other discrepancies

Step 1: harmonize by making all the files have the same columns (at least: subject codes (just the number), Scan code (subject codes and the year),
        visit code, and image code)

In [ ]:
dx_clean = pd.DataFrame({'SubjectCodes':dx['PTID'],'Diagnosis':dx['DIAGNOSIS']})
dx_clean = dx_clean.groupby('SubjectCodes').agg({'Diagnosis':'first'})
dx_clean.reset_index(inplace=True)
dx_clean.drop_duplicates('SubjectCodes',inplace=True)
dx_clean.dropna(inplace=True)
dx_clean['Diagnosis'] = dx_clean['Diagnosis'].apply(str)
replacements = {'1.0':'CN','2.0': 'MCI', '3.0': 'AD'}
dx_clean['Diagnosis'] = dx_clean['Diagnosis'].map(replacements).fillna(dx_clean['Diagnosis'])

dx_clean

,SubjectCodes,Diagnosis
0,002_S_0413,CN
1,002_S_1155,MCI
2,002_S_1261,MCI
3,002_S_1280,CN
4,002_S_4213,CN
...,...,...
1354,941_S_7074,CN
1355,941_S_7085,MCI
1356,941_S_7087,CN
1357,941_S_7091,CN


In [ ]:
subjcodes = []
for subj in imagelist['Code'].array:
    pattern = re.compile('\d{3}_\w_\d{4}')
    subjcodes.append(re.search(pattern,subj)[0])
imagecodes = []
for image in imagelist['Code'].array:
    pattern = re.compile('i\d*')
    imagecodes.append(re.search(pattern,image)[0])
visitcodes = []
for visit in imagelist['Code'].array:
    pattern = re.compile('y\d{2}')
    visitcodes.append(re.search(pattern,visit)[0])
imagelist.insert(1,'SubjectCodes',subjcodes)
imagelist.insert(2,'ImageCodes',imagecodes)
imagelist.insert(3,'VisitCodes',visitcodes)


In [ ]:
subjcodes = []
for subj in imagelistfsdn['Code'].array:
    pattern = re.compile('\d{3}_\w_\d{4}')
    subjcodes.append(re.search(pattern,subj)[0])
imagecodes = []
for image in imagelistfsdn['Code'].array:
    pattern = re.compile('i\d*')
    imagecodes.append(re.search(pattern,image)[0])
visitcodes = []
for visit in imagelistfsdn['Code'].array:
    pattern = re.compile('y\d{2}')
    visitcodes.append(re.search(pattern,visit)[0])
imagelistfsdn.insert(1,'SubjectCodes',subjcodes)
imagelistfsdn.insert(2,'ImageCodes',imagecodes)
imagelistfsdn.insert(3,'VisitCodes',visitcodes)
##imagelistfsdn

In [ ]:
subjcodes = []
for subj in reportraw['subjects'].array:
    pattern = re.compile('\d{3}_\w_\d{4}')
    subjcodes.append(re.search(pattern,subj)[0])
imagecodes = []
for image in reportraw['subjects'].array:
    pattern = re.compile('i\d*')
    imagecodes.append(re.search(pattern,image)[0])
visitcodes = []
for visit in reportraw['subjects'].array:
    pattern = re.compile('y\d{2}')
    visitcodes.append(re.search(pattern,visit)[0])
reportraw.insert(2,'SubjectCodes',subjcodes)
reportraw.insert(3,'ImageCodes',imagecodes)
reportraw.insert(4,'VisitCodes',visitcodes)

scancodes = 'ADNI3_' + reportraw['SubjectCodes'] + reportraw['VisitCodes']
reportraw.insert(2,'ScanCode',scancodes)
#reportraw

In [ ]:
subjcodes = []
for subj in report['subjects'].array:
    pattern = re.compile('\d{3}_\w_\d{4}')
    subjcodes.append(re.search(pattern,subj)[0])
imagecodes = []
for image in report['subjects'].array:
    pattern = re.compile('i\d*')
    imagecodes.append(re.search(pattern,image)[0])
visitcodes = []
for visit in report['subjects'].array:
    pattern = re.compile('y\d{2}')
    visitcodes.append(re.search(pattern,visit)[0])
report.insert(2,'SubjectCodes',subjcodes)
report.insert(3,'ImageCodes',imagecodes)
report.insert(4,'VisitCodes',visitcodes)

scancodes = 'ADNI3_' + report['SubjectCodes'] + report['VisitCodes']
report.insert(2,'ScanCode',scancodes)
#report

Step 2: make a deep copy so you don't screw things up when you merge

In [ ]:
imagelistfsdn_copy = copy.deepcopy(imagelistfsdn)
imagelist['check'] = 'yes'
imagelist

,Code,SubjectCodes,ImageCodes,VisitCodes,ScanCode,ImageType,ImageStore,ResearchGroup,check
0,ADNI3_002_S_0413y00_i863056,002_S_0413,i863056,y00,ADNI3_002_S_0413y00,T1,Dicom,CN,yes
1,ADNI3_002_S_0413y02_i1221051,002_S_0413,i1221051,y02,ADNI3_002_S_0413y02,T1,Dicom,CN,yes
2,ADNI3_002_S_1155y00_i843510,002_S_1155,i843510,y00,ADNI3_002_S_1155y00,T1,Dicom,MCI,yes
3,ADNI3_002_S_1155y01_i995496,002_S_1155,i995496,y01,ADNI3_002_S_1155y01,T1,Dicom,MCI,yes
4,ADNI3_002_S_1155y02_i1270004,002_S_1155,i1270004,y02,ADNI3_002_S_1155y02,T1,Dicom,MCI,yes
...,...,...,...,...,...,...,...,...,...
2130,ADNI3_941_S_7051y00_i1553005,941_S_7051,i1553005,y00,ADNI3_941_S_7051y00,T1,Dicom,CN,yes
2131,ADNI3_941_S_7074y00_i1588331,941_S_7074,i1588331,y00,ADNI3_941_S_7074y00,T1,Dicom,CN,yes
2132,ADNI3_941_S_7085y00_i1600180,941_S_7085,i1600180,y00,ADNI3_941_S_7085y00,T1,Dicom,MCI,yes
2133,ADNI3_941_S_7087y00_i1591321,941_S_7087,i1591321,y00,ADNI3_941_S_7087y00,T1,Dicom,CN,yes


In [ ]:
imagelistcopy = copy.deepcopy(imagelist)
reportcopy = copy.deepcopy(report)
reportcopy.drop_duplicates('subjects',inplace=True)    
imagelistcopy                 

,Code,SubjectCodes,ImageCodes,VisitCodes,ScanCode,ImageType,ImageStore,ResearchGroup,check
0,ADNI3_002_S_0413y00_i863056,002_S_0413,i863056,y00,ADNI3_002_S_0413y00,T1,Dicom,CN,yes
1,ADNI3_002_S_0413y02_i1221051,002_S_0413,i1221051,y02,ADNI3_002_S_0413y02,T1,Dicom,CN,yes
2,ADNI3_002_S_1155y00_i843510,002_S_1155,i843510,y00,ADNI3_002_S_1155y00,T1,Dicom,MCI,yes
3,ADNI3_002_S_1155y01_i995496,002_S_1155,i995496,y01,ADNI3_002_S_1155y01,T1,Dicom,MCI,yes
4,ADNI3_002_S_1155y02_i1270004,002_S_1155,i1270004,y02,ADNI3_002_S_1155y02,T1,Dicom,MCI,yes
...,...,...,...,...,...,...,...,...,...
2130,ADNI3_941_S_7051y00_i1553005,941_S_7051,i1553005,y00,ADNI3_941_S_7051y00,T1,Dicom,CN,yes
2131,ADNI3_941_S_7074y00_i1588331,941_S_7074,i1588331,y00,ADNI3_941_S_7074y00,T1,Dicom,CN,yes
2132,ADNI3_941_S_7085y00_i1600180,941_S_7085,i1600180,y00,ADNI3_941_S_7085y00,T1,Dicom,MCI,yes
2133,ADNI3_941_S_7087y00_i1591321,941_S_7087,i1591321,y00,ADNI3_941_S_7087y00,T1,Dicom,CN,yes


In [ ]:
reportrawcopy = copy.deepcopy(reportraw)
reportrawcopy.drop_duplicates('subjects',inplace=True)    
reportrawcopy

,subjects,research group,ScanCode,SubjectCodes,ImageCodes,VisitCodes,pvscount,pvsvol,icv norm,pvscountwm,pvsvolwm,icv norm wm,raw,WMH mask
0,ADNI3_114_S_2392y00_i909791,EMCI,ADNI3_114_S_2392y00,114_S_2392,i909791,y00,7767.0,16143.0,0.010479,6340.0,12022.0,0.007804,yes,yes
1,ADNI3_114_S_4404y00_i949873,EMCI,ADNI3_114_S_4404y00,114_S_4404,i949873,y00,5305.0,12342.0,0.009983,4129.0,8364.0,0.006765,yes,yes
2,ADNI3_128_S_4742y00_i1039209,EMCI,ADNI3_128_S_4742y00,128_S_4742,i1039209,y00,4057.0,15064.0,0.009716,2872.0,9966.0,0.006428,yes,yes
3,ADNI3_027_S_4919y04_i1398602,EMCI,ADNI3_027_S_4919y04,027_S_4919,i1398602,y04,3457.0,11502.0,0.008411,2782.0,7978.0,0.005834,yes,yes
4,ADNI3_067_S_2304y02_i1215232,EMCI,ADNI3_067_S_2304y02,067_S_2304,i1215232,y02,4856.0,13188.0,0.007504,3861.0,9106.0,0.005181,yes,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020,ADNI3_941_S_7074y00_i1588331,CN,ADNI3_941_S_7074y00,941_S_7074,i1588331,y00,4343.0,10179.0,0.007051,3410.0,6967.0,0.004826,yes,yes
2021,ADNI3_941_S_7085y00_i1600180,MCI,ADNI3_941_S_7085y00,941_S_7085,i1600180,y00,4362.0,9364.0,0.006991,3452.0,5785.0,0.004319,yes,yes
2022,ADNI3_941_S_7051y00_i1553005,CN,ADNI3_941_S_7051y00,941_S_7051,i1553005,y00,4365.0,14324.0,0.009672,3430.0,10342.0,0.006983,yes,yes
2023,ADNI3_941_S_7106y00_i1619403,MCI,ADNI3_941_S_7106y00,941_S_7106,i1619403,y00,3619.0,8365.0,0.005978,2479.0,4798.0,0.003429,yes,yes


Step 3: do your merging and investigating

Check which subjects are missing from the ADNI3_frangi list based on ADNI3_FSdn list

In [ ]:
mergedimagelist = imagelistfsdn_copy.merge(imagelist,how='left',on='ImageCodes')

In [ ]:
## this tells you which subjects are missing from ADNI3_frangi that don't match ADNI3_FSdn
## specifically, the last 3 don't match
nullmask = mergedimagelist.isnull().any(axis=1)
nullrow = mergedimagelist[nullmask]
nullrow

,Code_x,SubjectCodes_x,ImageCodes,VisitCodes_x,ScanCode_x,ImageType_x,ImageStore_x,Code_y,SubjectCodes_y,VisitCodes_y,ScanCode_y,ImageType_y,ImageStore_y,ResearchGroup,check
573,ADNI3_024_S_7025y00_i1515286,024_S_7025,i1515286,y00,ADNI3_024_S_7025y00,T1,Dicom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
978,ADNI3_051_S_6527y00_i1043951,051_S_6527,i1043951,y00,ADNI3_051_S_6527y00,T1,Dicom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1580,ADNI3_129_S_6482y00_i1021033,129_S_6482,i1021033,y00,ADNI3_129_S_6482y00,T1,Dicom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993,ADNI3_305_S_6263y00_i971096,305_S_6263,i971096,y00,ADNI3_305_S_6263y00,T1,Dicom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
nullrow.to_csv(os.path.join(path,'missingsubjects_fromimagelist.csv'))

Check which subjects are missing from the grand PVS report based on ADNI3_frangi list
- this will give you everything including those missing T1, WMH, and visit

In [ ]:
imagelistreport_merge = imagelistcopy.merge(reportcopy,how='left',on='ScanCode')
imagelistreport_merge.drop_duplicates('Code',inplace=True)
imagelistreport_merge

,Code,SubjectCodes_x,ImageCodes_x,VisitCodes_x,ScanCode,ImageType,ImageStore,ResearchGroup,check,subjects,...,ImageCodes_y,VisitCodes_y,pvscount,pvsvol,icv norm,pvscountwm,pvsvolwm,icv norm wm,raw,WMH mask
0,ADNI3_002_S_0413y00_i863056,002_S_0413,i863056,y00,ADNI3_002_S_0413y00,T1,Dicom,CN,yes,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ADNI3_002_S_0413y02_i1221051,002_S_0413,i1221051,y02,ADNI3_002_S_0413y02,T1,Dicom,CN,yes,ADNI3_002_S_0413y02_i1221051,...,i1221051,y02,1945.0,8300.0,0.005403,1302.0,4470.0,0.002910,no,yes
2,ADNI3_002_S_1155y00_i843510,002_S_1155,i843510,y00,ADNI3_002_S_1155y00,T1,Dicom,MCI,yes,ADNI3_002_S_1155y00_i843510,...,i843510,y00,738.0,3356.0,0.002123,210.0,660.0,0.000418,no,yes
3,ADNI3_002_S_1155y01_i995496,002_S_1155,i995496,y01,ADNI3_002_S_1155y01,T1,Dicom,MCI,yes,ADNI3_002_S_1155y01_i995496,...,i995496,y01,1988.0,7332.0,0.004549,1069.0,2793.0,0.001733,no,yes
4,ADNI3_002_S_1155y02_i1270004,002_S_1155,i1270004,y02,ADNI3_002_S_1155y02,T1,Dicom,MCI,yes,ADNI3_002_S_1155y02_i1270004,...,i1270004,y02,2020.0,8060.0,0.004978,1087.0,2856.0,0.001764,no,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2138,ADNI3_941_S_7051y00_i1553005,941_S_7051,i1553005,y00,ADNI3_941_S_7051y00,T1,Dicom,CN,yes,ADNI3_941_S_7051y00_i1553005,...,i1553005,y00,2010.0,10314.0,0.006964,1403.0,6060.0,0.004092,no,yes
2139,ADNI3_941_S_7074y00_i1588331,941_S_7074,i1588331,y00,ADNI3_941_S_7074y00,T1,Dicom,CN,yes,ADNI3_941_S_7074y00_i1588331,...,i1588331,y00,1509.0,6289.0,0.004356,969.0,2831.0,0.001961,no,yes
2140,ADNI3_941_S_7085y00_i1600180,941_S_7085,i1600180,y00,ADNI3_941_S_7085y00,T1,Dicom,MCI,yes,ADNI3_941_S_7085y00_i1600180,...,i1600180,y00,1607.0,4441.0,0.003315,954.0,1890.0,0.001411,no,yes
2141,ADNI3_941_S_7087y00_i1591321,941_S_7087,i1591321,y00,ADNI3_941_S_7087y00,T1,Dicom,CN,yes,ADNI3_941_S_7087y00_i1591321,...,i1591321,y00,1989.0,7926.0,0.004448,997.0,2721.0,0.001527,no,yes


In [ ]:
## this tells you which ones are missing, whether due to T1 missing, or WMH missing, or missing visit code, or some other reason
nullmask = imagelistreport_merge.isnull().any(axis=1)
nullrow = imagelistreport_merge[nullmask]
nullrow

,Code,SubjectCodes_x,ImageCodes_x,VisitCodes_x,ScanCode,ImageType,ImageStore,ResearchGroup,check,subjects,...,ImageCodes_y,VisitCodes_y,pvscount,pvsvol,icv norm,pvscountwm,pvsvolwm,icv norm wm,raw,WMH mask
0,ADNI3_002_S_0413y00_i863056,002_S_0413,i863056,y00,ADNI3_002_S_0413y00,T1,Dicom,CN,yes,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,ADNI3_002_S_5230y02_i1270084,002_S_5230,i1270084,y02,ADNI3_002_S_5230y02,T1,Dicom,SMC,yes,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59,ADNI3_003_S_4288y04_i1498579,003_S_4288,i1498579,y04,ADNI3_003_S_4288y04,T1,Dicom,CN,yes,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
114,ADNI3_003_S_6959y00_i1456305,003_S_6959,i1456305,y00,ADNI3_003_S_6959y00,T1,Dicom,CN,yes,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
116,ADNI3_003_S_7010y00_i1495816,003_S_7010,i1495816,y00,ADNI3_003_S_7010y00,T1,Dicom,CN,yes,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2114,ADNI3_941_S_6546y04_i1624954,941_S_6546,i1624954,y04,ADNI3_941_S_6546y04,T1,Dicom,CN,yes,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2126,ADNI3_941_S_6580y04_i1624283,941_S_6580,i1624283,y04,ADNI3_941_S_6580y04,T1,Dicom,CN,yes,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2128,ADNI3_941_S_6581y02_i1343754,941_S_6581,i1343754,y02,ADNI3_941_S_6581y02,T1,Dicom,CN,yes,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2131,ADNI3_941_S_6854y01_i1428390,941_S_6854,i1428390,y01,ADNI3_941_S_6854y01,T1,Dicom,AD,yes,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
nullrow.to_csv(os.path.join(path,'missingsubjectsfile.csv'))

Same thing but for raw grand report

In [ ]:
imagelistreport_merge = imagelistcopy.merge(reportrawcopy,how='left',on='ScanCode')
imagelistreport_merge.drop_duplicates('Code',inplace=True)
imagelistreport_merge

,Code,SubjectCodes_x,ImageCodes_x,VisitCodes_x,ScanCode,ImageType,ImageStore,ResearchGroup,check,subjects,...,ImageCodes_y,VisitCodes_y,pvscount,pvsvol,icv norm,pvscountwm,pvsvolwm,icv norm wm,raw,WMH mask
0,ADNI3_002_S_0413y00_i863056,002_S_0413,i863056,y00,ADNI3_002_S_0413y00,T1,Dicom,CN,yes,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ADNI3_002_S_0413y02_i1221051,002_S_0413,i1221051,y02,ADNI3_002_S_0413y02,T1,Dicom,CN,yes,ADNI3_002_S_0413y02_i1221051,...,i1221051,y02,3439.0,10102.0,0.006576,2512.0,6818.0,0.004438,yes,yes
2,ADNI3_002_S_1155y00_i843510,002_S_1155,i843510,y00,ADNI3_002_S_1155y00,T1,Dicom,MCI,yes,ADNI3_002_S_1155y00_i843510,...,i843510,y00,2383.0,9583.0,0.006062,1757.0,4950.0,0.003132,yes,yes
3,ADNI3_002_S_1155y01_i995496,002_S_1155,i995496,y01,ADNI3_002_S_1155y01,T1,Dicom,MCI,yes,ADNI3_002_S_1155y01_i995496,...,i995496,y01,3435.0,9084.0,0.005636,2409.0,5128.0,0.003182,yes,yes
4,ADNI3_002_S_1155y02_i1270004,002_S_1155,i1270004,y02,ADNI3_002_S_1155y02,T1,Dicom,MCI,yes,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2138,ADNI3_941_S_7051y00_i1553005,941_S_7051,i1553005,y00,ADNI3_941_S_7051y00,T1,Dicom,CN,yes,ADNI3_941_S_7051y00_i1553005,...,i1553005,y00,4365.0,14324.0,0.009672,3430.0,10342.0,0.006983,yes,yes
2139,ADNI3_941_S_7074y00_i1588331,941_S_7074,i1588331,y00,ADNI3_941_S_7074y00,T1,Dicom,CN,yes,ADNI3_941_S_7074y00_i1588331,...,i1588331,y00,4343.0,10179.0,0.007051,3410.0,6967.0,0.004826,yes,yes
2140,ADNI3_941_S_7085y00_i1600180,941_S_7085,i1600180,y00,ADNI3_941_S_7085y00,T1,Dicom,MCI,yes,ADNI3_941_S_7085y00_i1600180,...,i1600180,y00,4362.0,9364.0,0.006991,3452.0,5785.0,0.004319,yes,yes
2141,ADNI3_941_S_7087y00_i1591321,941_S_7087,i1591321,y00,ADNI3_941_S_7087y00,T1,Dicom,CN,yes,ADNI3_941_S_7087y00_i1591321,...,i1591321,y00,5078.0,12396.0,0.006957,3749.0,7351.0,0.004126,yes,yes


In [ ]:
## this tells you which ones are missing, whether due to T1 missing, or WMH missing, or missing visit code, or some other reason
nullmaskraw = imagelistreport_merge.isnull().any(axis=1)
nullrowraw = imagelistreport_merge[nullmaskraw]
nullrowraw

,Code,SubjectCodes_x,ImageCodes_x,VisitCodes_x,ScanCode,ImageType,ImageStore,ResearchGroup,check,subjects,...,ImageCodes_y,VisitCodes_y,pvscount,pvsvol,icv norm,pvscountwm,pvsvolwm,icv norm wm,raw,WMH mask
0,ADNI3_002_S_0413y00_i863056,002_S_0413,i863056,y00,ADNI3_002_S_0413y00,T1,Dicom,CN,yes,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ADNI3_002_S_1155y02_i1270004,002_S_1155,i1270004,y02,ADNI3_002_S_1155y02,T1,Dicom,MCI,yes,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,ADNI3_002_S_1261y00_i831065,002_S_1261,i831065,y00,ADNI3_002_S_1261y00,T1,Dicom,CN,yes,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,ADNI3_002_S_1261y01_i989320,002_S_1261,i989320,y01,ADNI3_002_S_1261y01,T1,Dicom,CN,yes,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,ADNI3_002_S_5230y02_i1270084,002_S_5230,i1270084,y02,ADNI3_002_S_5230y02,T1,Dicom,SMC,yes,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2114,ADNI3_941_S_6546y04_i1624954,941_S_6546,i1624954,y04,ADNI3_941_S_6546y04,T1,Dicom,CN,yes,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2126,ADNI3_941_S_6580y04_i1624283,941_S_6580,i1624283,y04,ADNI3_941_S_6580y04,T1,Dicom,CN,yes,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2128,ADNI3_941_S_6581y02_i1343754,941_S_6581,i1343754,y02,ADNI3_941_S_6581y02,T1,Dicom,CN,yes,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2131,ADNI3_941_S_6854y01_i1428390,941_S_6854,i1428390,y01,ADNI3_941_S_6854y01,T1,Dicom,AD,yes,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
nullrowraw.to_csv(os.path.join(path,'missingsubjectsfileRAW.csv'))

Figure out which of the nulls match the faulty image list

In [ ]:
faultysubjects

,subjects
0,ADNI3_141_S_6589y00_i1072748: missing raw flair
1,ADNI3_141_S_6589y00_i1072748: missing raw T1
2,ADNI3_141_S_6416y02_i1511356: missing raw flair
3,ADNI3_141_S_6416y02_i1511356: missing raw T1
4,ADNI3_141_S_6423y00_i1018387: missing raw flair
...,...
394,ADNI3_129_S_6146y02_i1277389: missing FSdn T1
395,ADNI3_131_S_6170y02_i1526394: missing FSdn T1
396,ADNI3_135_S_6411y04_i1591979: missing FSdn T1
397,ADNI3_135_S_6411y05_i10238728: missing FSdn T1


In [ ]:
subjcodes = []
for subj in faultysubjects['subjects'].array:
    pattern = re.compile('\d{3}_\w_\d{4}')
    subjcodes.append(re.search(pattern,subj)[0])
imagecodes = []
for image in faultysubjects['subjects'].array:
    pattern = re.compile('i\d*')
    imagecodes.append(re.search(pattern,image)[0])
visitcodes = []
for visit in faultysubjects['subjects'].array:
    pattern = re.compile('y\d{2}')
    visitcodes.append(re.search(pattern,visit)[0])

fullsubjcode = []
for c in faultysubjects['subjects'].array:
    pattern = re.compile('ADNI3_\d{3}_\w_\d{4}y\d{2}_i\d*')
    fullsubjcode.append(re.search(pattern,c)[0])


faultysubjects['SubjectCodes'] = subjcodes
faultysubjects['ImageCodes'] = imagecodes
faultysubjects['VisitCodes'] = visitcodes
faultysubjects['Code'] = fullsubjcode


faultysubjects.drop_duplicates('subjects',inplace=True)
faultysubjects

,subjects,SubjectCodes,ImageCodes,VisitCodes,Code
0,ADNI3_141_S_6589y00_i1072748: missing raw flair,141_S_6589,i1072748,y00,ADNI3_141_S_6589y00_i1072748
1,ADNI3_141_S_6589y00_i1072748: missing raw T1,141_S_6589,i1072748,y00,ADNI3_141_S_6589y00_i1072748
2,ADNI3_141_S_6416y02_i1511356: missing raw flair,141_S_6416,i1511356,y02,ADNI3_141_S_6416y02_i1511356
3,ADNI3_141_S_6416y02_i1511356: missing raw T1,141_S_6416,i1511356,y02,ADNI3_141_S_6416y02_i1511356
4,ADNI3_141_S_6423y00_i1018387: missing raw flair,141_S_6423,i1018387,y00,ADNI3_141_S_6423y00_i1018387
...,...,...,...,...,...
394,ADNI3_129_S_6146y02_i1277389: missing FSdn T1,129_S_6146,i1277389,y02,ADNI3_129_S_6146y02_i1277389
395,ADNI3_131_S_6170y02_i1526394: missing FSdn T1,131_S_6170,i1526394,y02,ADNI3_131_S_6170y02_i1526394
396,ADNI3_135_S_6411y04_i1591979: missing FSdn T1,135_S_6411,i1591979,y04,ADNI3_135_S_6411y04_i1591979
397,ADNI3_135_S_6411y05_i10238728: missing FSdn T1,135_S_6411,i10238728,y05,ADNI3_135_S_6411y05_i10238728


In [ ]:
faultysubjectscopy = copy.deepcopy(faultysubjects)

In [ ]:
faultysubjectmerge = faultysubjectscopy.merge(nullrow,how='left',on='Code')
faultysubjectmerge

,subjects_x,SubjectCodes,ImageCodes,VisitCodes,Code,SubjectCodes_x,ImageCodes_x,VisitCodes_x,ScanCode,ImageType,...,ImageCodes_y,VisitCodes_y,pvscount,pvsvol,icv norm,pvscountwm,pvsvolwm,icv norm wm,raw,WMH mask
0,ADNI3_141_S_6589y00_i1072748: missing raw flair,141_S_6589,i1072748,y00,ADNI3_141_S_6589y00_i1072748,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ADNI3_141_S_6589y00_i1072748: missing raw T1,141_S_6589,i1072748,y00,ADNI3_141_S_6589y00_i1072748,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ADNI3_141_S_6416y02_i1511356: missing raw flair,141_S_6416,i1511356,y02,ADNI3_141_S_6416y02_i1511356,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ADNI3_141_S_6416y02_i1511356: missing raw T1,141_S_6416,i1511356,y02,ADNI3_141_S_6416y02_i1511356,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ADNI3_141_S_6423y00_i1018387: missing raw flair,141_S_6423,i1018387,y00,ADNI3_141_S_6423y00_i1018387,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,ADNI3_129_S_6146y02_i1277389: missing FSdn T1,129_S_6146,i1277389,y02,ADNI3_129_S_6146y02_i1277389,129_S_6146,i1277389,y02,ADNI3_129_S_6146y02,T1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
395,ADNI3_131_S_6170y02_i1526394: missing FSdn T1,131_S_6170,i1526394,y02,ADNI3_131_S_6170y02_i1526394,131_S_6170,i1526394,y02,ADNI3_131_S_6170y02,T1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
396,ADNI3_135_S_6411y04_i1591979: missing FSdn T1,135_S_6411,i1591979,y04,ADNI3_135_S_6411y04_i1591979,135_S_6411,i1591979,y04,ADNI3_135_S_6411y04,T1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
397,ADNI3_135_S_6411y05_i10238728: missing FSdn T1,135_S_6411,i10238728,y05,ADNI3_135_S_6411y05_i10238728,135_S_6411,i10238728,y05,ADNI3_135_S_6411y05,T1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# gives you the subjects that are faulty but not in the missing list
nullrowsfaultysubjects = faultysubjectmerge['SubjectCodes_x'].isnull()
nullrowsfaultysubjects_nomatch = faultysubjectmerge[nullrowsfaultysubjects]
#nullrowsfaultysubjects_nomatch
# nullrowsfaultysubjects_nomatch

In [ ]:
nullrowsfaultysubjects_nomatch.to_csv(os.path.join(path,'nomatch_faultymissing.csv'))

In [ ]:
# gives you the subjects that are in the faulty subjects list AND missing subjects list
nullrowsfaultysubjects_not = faultysubjectmerge['SubjectCodes_x'].notnull()
nullrowsfaultysubjects_match = faultysubjectmerge[nullrowsfaultysubjects_not]
nullrowsfaultysubjects_match.head()

,subjects_x,SubjectCodes,ImageCodes,VisitCodes,Code,SubjectCodes_x,ImageCodes_x,VisitCodes_x,ScanCode,ImageType,...,ImageCodes_y,VisitCodes_y,pvscount,pvsvol,icv norm,pvscountwm,pvsvolwm,icv norm wm,raw,WMH mask
376,ADNI3_003_S_6959y00_i1456305: missing FSdn T1,003_S_6959,i1456305,y00,ADNI3_003_S_6959y00_i1456305,003_S_6959,i1456305,y00,ADNI3_003_S_6959y00,T1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
377,ADNI3_005_S_6084y00_i915206: missing FSdn T1,005_S_6084,i915206,y00,ADNI3_005_S_6084y00_i915206,005_S_6084,i915206,y00,ADNI3_005_S_6084y00,T1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
378,ADNI3_007_S_2394y02_i1335884: missing FSdn T1,007_S_2394,i1335884,y02,ADNI3_007_S_2394y02_i1335884,007_S_2394,i1335884,y02,ADNI3_007_S_2394y02,T1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
379,ADNI3_009_S_0751y05_i1593548: missing FSdn T1,009_S_0751,i1593548,y05,ADNI3_009_S_0751y05_i1593548,009_S_0751,i1593548,y05,ADNI3_009_S_0751y05,T1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
380,ADNI3_022_S_6847y00_i1283754: missing FSdn T1,022_S_6847,i1283754,y00,ADNI3_022_S_6847y00_i1283754,022_S_6847,i1283754,y00,ADNI3_022_S_6847y00,T1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
nullrowsfaultysubjects_match.to_csv(os.path.join(path,'mergefaultysubjectsmissingsubjects.csv'))